In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token = HF_TOKEN)

In [3]:
!pip install -q -U unsloth
!pip install -q evaluate rouge_score sacrebleu bert_score

from unsloth import FastLanguageModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-07-25 15:24:46.362314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753457086.731912      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753457086.849933      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model_name = "te4bag/GRIT-Full-alpaca-llama-3.2-3B-Energy-0.9"

# Load model with tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,              # "auto" by default; set to torch.float16 or bfloat16 if needed
    load_in_4bit = True        # for quantized inference (efficient)
)

# Speed up with Flash Attention 2 (optional)
FastLanguageModel.for_inference(model)


/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'qalora_group_size', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_qalora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'qalora_group_size', 'trainable_token_indices', 'use_qalora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.7.8 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, max_new_tokens=100)

# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response)


In [6]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
],return_tensors = "pt").to("cuda")

import torch
with torch.no_grad():                   # turn gradients off
    output_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        temperature=0.0,

        num_beams = 1,
        
        use_cache=True, 
        pad_token_id=tokenizer.eos_token_id
    )
generated_text =  tokenizer.batch_decode(output_ids)

# Extract only what's after "### Response:"
if "### Response:" in generated_text[0]:
    response = generated_text[0].split("### Response:")[1].strip()
else:
    response = generated_text[0].strip()

print("Extracted response:\n" + response)

Extracted response:
13, 21, 34, 55, 89, 144<|end_of_text|>


In [7]:
# # 1. Create a small evaluation dataset with coding prompts
# eval_dataset = [
#     {
#         "instruction": "Write a Python function to check if a number is a palindrome.",
#         "input": "The function should take an integer and return True if it's a palindrome, False otherwise.",
#         "reference": "def is_palindrome(n):\n    return str(n) == str(n)[::-1]"
#     },
#     {
#         "instruction": "Complete the Python function to calculate the factorial of a number.",
#         "input": "def factorial(n):\n    # Add your code here",
#         "reference": "def factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)"
#     },
#     {
#         "instruction": "Translate the following Python list comprehension to a regular for loop.",
#         "input": "squares = [x*x for x in range(10)]",
#         "reference": "squares = []\nfor x in range(10):\n    squares.append(x*x)"
#     }
# ]

# # 2. Helper function to generate responses from your model
# def generate_responses(dataset, model, tokenizer):
#     predictions = []
#     alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# """
#     print("Generating model predictions...")
#     for item in dataset:
#         prompt = alpaca_prompt_template.format(item["instruction"], item["input"])
#         inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        
#         # outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, pad_token_id=tokenizer.eos_token_id)
#         with torch.no_grad():                   # turn gradients off
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=256,
#                 do_sample=False,
#                 temperature=0.0,
#                 num_beams = 1,
#                 use_cache=True, 
#                 pad_token_id=tokenizer.eos_token_id
#             )
#         full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         try:
#             prediction = full_response.split("### Response:")[1].strip()
#         except IndexError:
#             prediction = ""
        
#         predictions.append(prediction)
#         print(f"Instruction: {item['instruction']}\n- Model Output: {prediction}\n---")
        
#     return predictions

# # 3. Generate the predictions and references (This creates the missing variables)
# predictions = generate_responses(eval_dataset, model, tokenizer)
# references = [item['reference'] for item in eval_dataset]

# print("\n\n--- Predictions and References have been successfully generated! ---")
# print("--- Now calculating scores... ---\n")

In [8]:
from datasets import load_dataset
ds = load_dataset("tatsu-lab/alpaca", split="train")
# If your GPU cannot fit the whole set, take a random 1 k-sample, e.g.:
ds = ds.shuffle(seed=42).select(range(1000))



# 2. Helper function to generate responses from your model
def generate_responses(dataset, model, tokenizer):
    predictions = []
    references = []
    alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""
    print("Generating model predictions...")
    for item in dataset:
        prompt = alpaca_prompt_template.format(item["instruction"], item["input"])
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        
        # outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, pad_token_id=tokenizer.eos_token_id)
        with torch.no_grad():                   # turn gradients off
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=False,
                temperature=0.0,
                num_beams = 1,
                use_cache=True, 
                pad_token_id=tokenizer.eos_token_id
            )
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        try:
            prediction = full_response.split("### Response:")[1].strip()
        except IndexError:
            prediction = ""
        
        predictions.append(prediction)
        references.append(item["output"])
        # print(f"Instruction: {item['instruction']}\n- Model Output: {prediction}\n---")
        
    return predictions, references

# 3. Generate the predictions and references (This creates the missing variables)
predictions, references = generate_responses(ds, model, tokenizer) 

print("\n\n--- Predictions and References have been successfully generated! ---")
print("--- Now calculating scores... ---\n")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Generating model predictions...


--- Predictions and References have been successfully generated! ---
--- Now calculating scores... ---



In [9]:

import evaluate

# --- BLEU Score ---
bleu_metric = evaluate.load("bleu")
# BLEU expects a list of lists for references
bleu_results = bleu_metric.compute(predictions=predictions, references=[[r] for r in references])
print("--- BLEU Score ---")
print(f"BLEU: {bleu_results['bleu']:.4f}")
print(f"Precisions: {bleu_results['precisions']}")


# --- ROUGE Score ---
rouge_metric = evaluate.load("rouge")
rouge_results = rouge_metric.compute(predictions=predictions, references=references)
print("\n--- ROUGE Score ---")
print(f"ROUGE-1: {rouge_results['rouge1']:.4f} (unigram overlap)")
print(f"ROUGE-2: {rouge_results['rouge2']:.4f} (bigram overlap)")
print(f"ROUGE-L: {rouge_results['rougeL']:.4f} (longest common subsequence)")

--- BLEU Score ---
BLEU: 0.1233
Precisions: [0.3430065859210732, 0.15024669743752983, 0.0847369178723098, 0.052856174795333416]



--- ROUGE Score ---
ROUGE-1: 0.4355 (unigram overlap)
ROUGE-2: 0.2240 (bigram overlap)
ROUGE-L: 0.3632 (longest common subsequence)


In [10]:
#
# ADD THIS CELL TO YOUR NOTEBOOK (Replaces the previous BERT-Score cell)
#

# Install the necessary library (it's good practice to keep it in the cell)
!pip install -q bert_score

import evaluate

# Correctly load the "bertscore" metric (no underscore)
bertscore_metric = evaluate.load("bertscore")
bert_results = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

# We get an average over all the samples
avg_precision = sum(bert_results['precision']) / len(bert_results['precision'])
avg_recall = sum(bert_results['recall']) / len(bert_results['recall'])
avg_f1 = sum(bert_results['f1']) / len(bert_results['f1'])

print("\n--- BERT-Score ---")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall:    {avg_recall:.4f}")
print(f"Average F1 Score:  {avg_f1:.4f}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- BERT-Score ---
Average Precision: 0.9041
Average Recall:    0.8963
Average F1 Score:  0.8998
